In [1]:
# import neccessary libraries

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# data visualization setting

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)

In [3]:
# load the dataset using pd

df = pd.read_excel("datasets/Insurance_fraud.xlsx")
df.to_csv("Insurance_fraud.csv", index=False)